In [6]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

# Charger le dataset
df_train = pd.read_csv('train.csv')
df = df_train

# Ajouter une colonne pour le nom court
df['Name_short'] = df['Name'].str.split(' ').str[0]

# Créer l'application Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Définir la mise en page de l'application
app.layout = html.Div([
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='fuel-type-dropdown',
                options=[{'label': x, 'value': x} for x in df['Fuel_Type'].unique()],
                value='All Fuel Types',
                clearable=False
            ),
            dcc.Dropdown(
                id='start-year-dropdown',
                options=[{'label': x, 'value': x} for x in sorted(df['Year'].unique(), reverse=True)],
                value=df['Year'].max(),
                clearable=False
            ),
            dcc.Dropdown(
                id='end-year-dropdown',
                options=[{'label': x, 'value': x} for x in sorted(df['Year'].unique(), reverse=True)],
                value=df['Year'].min(),
                clearable=False
            ),
            dcc.Dropdown(
                id='name-dropdown',
                options=[{'label': x, 'value': x} for x in df['Name_short'].unique()],
                value='All Names',
                clearable=False
            )
        ], width=3),
        dbc.Col([
            dcc.Graph(id='price-graph')
        ], width=9)
    ])
])

# Définir la fonction pour mettre à jour le graphique
def update_graph(fuel_type, start_year, end_year, name):
    mask = (df['Fuel_Type'] == fuel_type) if fuel_type != 'All Fuel Types' else np.full(len(df), True)
    mask &= (df['Year'].between(start_year, end_year)) if start_year <= end_year else np.full(len(df), False)
    mask &= (df['Name_short'] == name) if name != 'All Names' else np.full(len(df), True)

    df_filtered = df[mask]

    # Convertir les prix en unités de lakhs en roupies indiennes
    conversion_rate = 87
    lakh_conversion = 100000
    df_filtered['Price_Rupie'] = df_filtered['Price'] * lakh_conversion

    # Convertir les prix en euros
    df_filtered['Price_EUR'] = df_filtered['Price_Rupie'] / conversion_rate
    df_filtered['Price_EUR'] = df_filtered['Price_EUR'].round(2)

    # Calculer la moyenne des prix par année
    avg_price_by_year = df_filtered.groupby('Year')['Price_EUR'].mean().reset_index()

    fig = px.line(avg_price_by_year, x='Year', y='Price_EUR', title='Price Evolution by Fuel Type, Year, and Name')
    return fig

# Définir le callback pour mettre à jour le graphique
@app.callback(
    Output('price-graph', 'figure'),
    Input('fuel-type-dropdown', 'value'),
    Input('start-year-dropdown', 'value'),
    Input('end-year-dropdown', 'value'),
    Input('name-dropdown', 'value')
)
def display_graph(fuel_type, start_year, end_year, name):
    return update_graph(fuel_type, start_year, end_year, name)

# Exécuter l'application
if __name__ == '__main__':
    app.run_server(debug=True, port=8053)


C:\Users\berlu\AppData\Local\Temp\ipykernel_24920\1421655110.py:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\berlu\AppData\Local\Temp\ipykernel_24920\1421655110.py:67: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\berlu\AppData\Local\Temp\ipykernel_24920\1421655110.py:68: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa